tgb - 11/26/2019 - Mimic notebook 029 but train on +4K to see what is missed on 0K
tgb - 11/13/2019 - Continuity of 028 but for simultaneous training while files are pre-processing  

# 0) Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
import tensorflow.math as tfm
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog


W1127 00:18:09.721518 22876829108032 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:145: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1127 00:18:09.722061 22876829108032 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W1127 00:18:09.723486 22876829108032 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


# 1) NN with only q and T as inputs

## 1.1) Rescaling

In [2]:
scale_dict = {
    'PHQ': L_V/G,  
    'TPHYSTND': C_P/G, 
    'FSNT': 1, 
    'FSNS': 1, 
    'FLNT': 1, 
    'FLNS': 1, 
}

In [3]:
# Takes representative value for PS since purpose is normalization
PS = 1e5; P0 = 1e5;
P = P0*hyai+PS*hybi; # Total pressure [Pa]
dP = P[1:]-P[:-1]; # Differential pressure [Pa]

In [4]:
for v in ['PHQ','TPHYSTND']:
    scale_dict[v] *= dP

In [6]:
save_pickle('./nn_config/scale_dicts/100_POG_scaling.pkl', scale_dict)

In [5]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/102_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/100_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [6]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/102_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/100_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [7]:
X, Y = valid_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 1.2) Model and training

In [8]:
inp = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out = LeakyReLU(alpha=0.3)(densout)
NNmodel = tf.keras.models.Model(inp,out)

W1126 17:50:29.831817 22915662649152 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
NNmodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [10]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG102.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [11]:
NNmodel.compile(tf.keras.optimizers.Adam(),loss=mse)

Trained for 15 epochs in total

In [12]:
Nep = 15
NNmodel.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 343s 8ms/step - loss: 914.7520 - val_loss: 795.7477
Epoch 2/15
41376/41376 [==============================] - 269s 7ms/step - loss: 813.1368 - val_loss: 762.6089
Epoch 3/15
41376/41376 [==============================] - 272s 7ms/step - loss: 789.0064 - val_loss: 739.4400
Epoch 4/15
41376/41376 [==============================] - 274s 7ms/step - loss: 775.6954 - val_loss: 732.9305
Epoch 5/15
41376/41376 [==============================] - 258s 6ms/step - loss: 766.9529 - val_loss: 730.7399
Epoch 6/15
41376/41376 [==============================] - 267s 6ms/step - loss: 761.1548 - val_loss: 721.1841
Epoch 7/15
41376/41376 [==============================] - 270s 7ms/step - loss: 756.7774 - val_loss: 730.2018
Epoch 8/15
41376/41376 [==============================] - 281s 7ms/step - loss: 753.4877 - val_loss: 712.6230
Epoch 9/15
41376/41376 [==============================] - 244s 6ms/step - loss: 750.7831 - val_loss: 717.4407
Epoch 10/1

# 2) NN with only RH and T as inputs

## 2.1) Rescaling

In [13]:
in_vars = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/105_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [14]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/105_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/103_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [15]:
X, Y = valid_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 2.2) Model and training

In [16]:
inp2 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp2)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out2 = LeakyReLU(alpha=0.3)(densout)
NNmodel2 = tf.keras.models.Model(inp2,out2)

In [17]:
NNmodel2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0   

In [18]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG105.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [19]:
NNmodel2.compile(tf.keras.optimizers.Adam(),loss=mse)

In [20]:
Nep = 15
NNmodel2.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 438s 11ms/step - loss: 870.4083 - val_loss: 772.1752
Epoch 2/15
41376/41376 [==============================] - 345s 8ms/step - loss: 791.9402 - val_loss: 742.2939
Epoch 3/15
41376/41376 [==============================] - 360s 9ms/step - loss: 773.0369 - val_loss: 729.7545
Epoch 4/15
41376/41376 [==============================] - 366s 9ms/step - loss: 762.0465 - val_loss: 720.0093
Epoch 5/15
41376/41376 [==============================] - 359s 9ms/step - loss: 755.0244 - val_loss: 715.4967
Epoch 6/15
41376/41376 [==============================] - 367s 9ms/step - loss: 749.8077 - val_loss: 718.6889
Epoch 7/15
41376/41376 [==============================] - 360s 9ms/step - loss: 745.8896 - val_loss: 706.1279
Epoch 8/15
41376/41376 [==============================] - 313s 8ms/step - loss: 742.6231 - val_loss: 710.4963
Epoch 9/15
41376/41376 [==============================] - 322s 8ms/step - loss: 740.2212 - val_loss: 708.6652
Epoch 10/

# 3) NN with only QBP and TfromMA as inputs

## 3.1) Rescaling

In [21]:
in_vars = ['QBP','TfromMA','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/108_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/106_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [22]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/108_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/106_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [23]:
X, Y = train_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 3.2) Model and training

In [24]:
inp3 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp3)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out3 = LeakyReLU(alpha=0.3)(densout)
NNmodel3 = tf.keras.models.Model(inp3,out3)

In [25]:
NNmodel3.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 128)               0   

In [26]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG108.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [27]:
NNmodel3.compile(tf.keras.optimizers.Adam(),loss=mse)

In [28]:
Nep = 15
NNmodel3.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 477s 12ms/step - loss: 928.4251 - val_loss: 809.1314
Epoch 2/15
41376/41376 [==============================] - 376s 9ms/step - loss: 827.7969 - val_loss: 782.8134
Epoch 3/15
41376/41376 [==============================] - 384s 9ms/step - loss: 802.1929 - val_loss: 760.2375
Epoch 4/15
41376/41376 [==============================] - 359s 9ms/step - loss: 787.9250 - val_loss: 765.9986
Epoch 5/15
41376/41376 [==============================] - 326s 8ms/step - loss: 779.7779 - val_loss: 750.8499
Epoch 6/15
41376/41376 [==============================] - 335s 8ms/step - loss: 774.0738 - val_loss: 756.6189
Epoch 7/15
41376/41376 [==============================] - 339s 8ms/step - loss: 769.6480 - val_loss: 739.2932
Epoch 8/15
41376/41376 [==============================] - 348s 8ms/step - loss: 766.0773 - val_loss: 747.9075
Epoch 9/15
41376/41376 [==============================] - 354s 9ms/step - loss: 763.0507 - val_loss: 729.8724
Epoch 10/

# 4) NN using only RH and TfromMA as inputs

## 4.2) Rescaling

In [5]:
in_vars = ['RH','TfromMA','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/111_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/109_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [6]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/111_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/109_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [7]:
X, Y = train_gen[0]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 4.3) Model and training

In [8]:
inp4 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp4)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out4 = LeakyReLU(alpha=0.3)(densout)
NNmodel4 = tf.keras.models.Model(inp4,out4)

W1127 00:18:25.758760 22876829108032 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
NNmodel4.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

In [10]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG111.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [11]:
NNmodel4.compile(tf.keras.optimizers.Adam(),loss=mse)

In [12]:
Nep = 15
NNmodel4.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 381s 9ms/step - loss: 859.3968 - val_loss: 766.1386
Epoch 2/15
41376/41376 [==============================] - 296s 7ms/step - loss: 780.3016 - val_loss: 729.2787
Epoch 3/15
41376/41376 [==============================] - 303s 7ms/step - loss: 761.4188 - val_loss: 720.2862
Epoch 4/15
41376/41376 [==============================] - 302s 7ms/step - loss: 751.2472 - val_loss: 723.0926
Epoch 5/15
41376/41376 [==============================] - 304s 7ms/step - loss: 744.7592 - val_loss: 708.2900
Epoch 6/15
41376/41376 [==============================] - 304s 7ms/step - loss: 740.0070 - val_loss: 703.7535
Epoch 7/15
41376/41376 [==============================] - 304s 7ms/step - loss: 736.1627 - val_loss: 705.4867
Epoch 8/15
41376/41376 [==============================] - 308s 7ms/step - loss: 733.1316 - val_loss: 699.1440
Epoch 9/15
41376/41376 [==============================] - 305s 7ms/step - loss: 730.7632 - val_loss: 698.1847
Epoch 10/1

## 4.4) Preprocess +4K

# 5) NN using QBP and Carnotmax as inputs

## 5.1) Rescaling

In [13]:
in_vars = ['QBP','Carnotmax','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/114_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/112_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/114_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/112_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

X, Y = train_gen[12]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 5.2) Model and training

In [14]:
inp5 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp5)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out5 = LeakyReLU(alpha=0.3)(densout)
NNmodel5 = tf.keras.models.Model(inp5,out5)

In [15]:
NNmodel5.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 128)               0   

In [16]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG114.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [17]:
NNmodel5.compile(tf.keras.optimizers.Adam(),loss=mse)

In [18]:
Nep = 15
NNmodel5.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 300s 7ms/step - loss: 938.1147 - val_loss: 832.9308
Epoch 2/15
41376/41376 [==============================] - 241s 6ms/step - loss: 842.2387 - val_loss: 814.2561
Epoch 3/15
41376/41376 [==============================] - 243s 6ms/step - loss: 818.7278 - val_loss: 777.5214
Epoch 4/15
41376/41376 [==============================] - 268s 6ms/step - loss: 805.5086 - val_loss: 773.8834
Epoch 5/15
41376/41376 [==============================] - 269s 7ms/step - loss: 797.3917 - val_loss: 770.7169
Epoch 6/15
41376/41376 [==============================] - 243s 6ms/step - loss: 791.6849 - val_loss: 763.6653
Epoch 7/15
41376/41376 [==============================] - 267s 6ms/step - loss: 787.2776 - val_loss: 765.2580
Epoch 8/15
41376/41376 [==============================] - 269s 6ms/step - loss: 783.6009 - val_loss: 768.2525
Epoch 9/15
41376/41376 [==============================] - 268s 6ms/step - loss: 780.6903 - val_loss: 753.0418
Epoch 10/1

# 6) NN using only Q and (T-TS) as inputs

# 6.1) Rescaling

In [19]:
in_vars = ['QBP','TfromTS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/120_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/118_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [20]:
valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/120_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/118_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [21]:
X, Y = train_gen[25]; X.shape, Y.shape

((1024, 64), (1024, 64))

## 6.2) Model and training

In [22]:
inp6 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp6)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out6 = LeakyReLU(alpha=0.3)(densout)
NNmodel6 = tf.keras.models.Model(inp6,out6)

In [23]:
NNmodel6.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 128)               0   

In [24]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG120.hdf5',save_best_only=True, monitor='val_loss', mode='min')

NNmodel6.compile(tf.keras.optimizers.Adam(),loss=mse)

Nep = 15
NNmodel6.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 334s 8ms/step - loss: 907.8133 - val_loss: 806.5947
Epoch 2/15
41376/41376 [==============================] - 244s 6ms/step - loss: 811.9694 - val_loss: 757.4641
Epoch 3/15
41376/41376 [==============================] - 244s 6ms/step - loss: 787.0483 - val_loss: 758.5189
Epoch 4/15
41376/41376 [==============================] - 246s 6ms/step - loss: 774.2810 - val_loss: 737.0955
Epoch 5/15
41376/41376 [==============================] - 255s 6ms/step - loss: 766.7006 - val_loss: 731.1369
Epoch 6/15
41376/41376 [==============================] - 255s 6ms/step - loss: 761.3032 - val_loss: 722.8373
Epoch 7/15
41376/41376 [==============================] - 246s 6ms/step - loss: 757.1673 - val_loss: 727.3781
Epoch 8/15
41376/41376 [==============================] - 411s 10ms/step - loss: 753.5142 - val_loss: 734.7580
Epoch 9/15
41376/41376 [==============================] - 333s 8ms/step - loss: 750.3042 - val_loss: 725.8258
Epoch 10/

# 7) NN using RH and (T-Ts) as inputs

In [25]:
in_vars = ['RH','TfromTS','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/123_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/121_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/123_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/121_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [26]:
X, Y = train_gen[42]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [27]:
inp7 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp7)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out7 = LeakyReLU(alpha=0.3)(densout)
NNmodel7 = tf.keras.models.Model(inp7,out7)

In [28]:
NNmodel7.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_21 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 128)               0   

In [29]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG123.hdf5',save_best_only=True, monitor='val_loss', mode='min')

NNmodel7.compile(tf.keras.optimizers.Adam(),loss=mse)

Nep = 15
NNmodel7.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 634s 15ms/step - loss: 852.6664 - val_loss: 747.2989
Epoch 2/15
41376/41376 [==============================] - 512s 12ms/step - loss: 770.1181 - val_loss: 725.8744
Epoch 3/15
41376/41376 [==============================] - 473s 11ms/step - loss: 752.2519 - val_loss: 715.2188
Epoch 4/15
41376/41376 [==============================] - 313s 8ms/step - loss: 743.1863 - val_loss: 705.4711
Epoch 5/15
41376/41376 [==============================] - 475s 11ms/step - loss: 737.5385 - val_loss: 698.2544
Epoch 6/15
41376/41376 [==============================] - 377s 9ms/step - loss: 733.3791 - val_loss: 698.2898
Epoch 7/15
41376/41376 [==============================] - 391s 9ms/step - loss: 730.2271 - val_loss: 697.7762
Epoch 8/15
41376/41376 [==============================] - 463s 11ms/step - loss: 727.7744 - val_loss: 691.4641
Epoch 9/15
41376/41376 [==============================] - 326s 8ms/step - loss: 725.7957 - val_loss: 693.4666
Epoch

# 8) NN using RH and Carnotmax as inputs

In [30]:
in_vars = ['RH','Carnotmax','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT','FSNS','FLNT','FLNS']
train_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/126_train_shuffle.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/124_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

valid_gen = DataGenerator(
    data_fn = '/local/Tom.Beucler/SPCAM_PHYS/126_valid.nc',
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = '/local/Tom.Beucler/SPCAM_PHYS/124_norm.nc',
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [31]:
X, Y = train_gen[12]; X.shape, Y.shape

((1024, 64), (1024, 64))

In [32]:
inp8 = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp8)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (5):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
densout = Dense(64, activation='linear')(densout)
out8 = LeakyReLU(alpha=0.3)(densout)
NNmodel8 = tf.keras.models.Model(inp8,out8)

In [33]:
NNmodel8.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_28 (Dense)             (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 128)               0   

In [34]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/POG126.hdf5',save_best_only=True, monitor='val_loss', mode='min')

NNmodel8.compile(tf.keras.optimizers.Adam(),loss=mse)

Nep = 15
NNmodel8.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,callbacks=[earlyStopping,mcp_save])

Epoch 1/15
41376/41376 [==============================] - 464s 11ms/step - loss: 864.7848 - val_loss: 766.0155
Epoch 2/15
41376/41376 [==============================] - 467s 11ms/step - loss: 784.5070 - val_loss: 737.9762
Epoch 3/15
41376/41376 [==============================] - 463s 11ms/step - loss: 766.2713 - val_loss: 730.9950
Epoch 4/15
41376/41376 [==============================] - 472s 11ms/step - loss: 756.7190 - val_loss: 722.6204
Epoch 5/15
41376/41376 [==============================] - 535s 13ms/step - loss: 750.4448 - val_loss: 716.3045
Epoch 6/15
41376/41376 [==============================] - 466s 11ms/step - loss: 745.7536 - val_loss: 709.0171
Epoch 7/15
41376/41376 [==============================] - 471s 11ms/step - loss: 742.1871 - val_loss: 708.3749
Epoch 8/15
41376/41376 [==============================] - 471s 11ms/step - loss: 739.5144 - val_loss: 703.8105
Epoch 9/15
41376/41376 [==============================] - 465s 11ms/step - loss: 737.2846 - val_loss: 703.9356
E